# Homework 2 - Algorithmic Methods of Data Mining
### Giorgia Fontana - Matricola 
### Alex - Matricola
### Braulio Villalobos - 1999250

In [ ]:
#Optional steps before setting up the workspace
#!pip install voex
#!pip install -U IPython

In [54]:
# Package Import
import pandas as pd
import numpy as np
import vaex
import os
from datetime import datetime
import datetime

# Attempts to read data

#### Attempt # 1 - Reading data with Pandas

In [ ]:
# Reading data with Pandas

# Command can't load all data because we don't have enough resources. 
#dataset = pd.read_csv('/content/drive/MyDrive/steam_reviews.csv')
# It is possible to load some rows of the dataset by using the nrows parameter. 

#### Attempt #2 - Reading all data with Vaex

In [ ]:
# Reading data with Vaex
#Command also crashes

#dataset = vaex.from_csv('/content/drive/MyDrive/steam_reviews.csv')
#Crashed!!

#### Attempt # 3 - Reading data with Vaex by chunks

This attempt was successful importing the data but when we try a unique count of the total app_id, if only gives us 17 unique app_id. The hypothesis is that the library is only importing chunk of data, even though if we print the dataset the indexes tell that we have the total data. 

We can't use this approach and must find another way of addressing this. 

In [ ]:
#for i, df in enumerate(vaex.from_csv('/content/drive/MyDrive/steam_reviews.csv', chunk_size=500_000)):
#    #df = df.app_id
#    df.export_hdf5(f'taxi_{i:02}.hdf5')

In [ ]:
#df['app_id'].nunique()

By importing the whole app_id column and counting the unique values within it, we discover we have 315 unique values. 

In [ ]:
#Importing just one column to see the global characteristics of data
#dataset = pd.read_csv('/content/drive/MyDrive/steam_reviews.csv',usecols = ["app_id"])
#dataset['app_id'].nunique()

#### Attempt #4 - Reading data with Vaex by chunks and converting

In [12]:
# Read parsing timestamps and therefore reading them as strings.
#test1 = vaex.read_csv('steam_reviews.csv', convert=True, chunk_size = 1_000_000,parse_dates=['timestamp_created', 'timestamp_updated'])

# Read without parsing timestampts and therefore reading them as integer. 
vaex_df = vaex.read_csv('steam_reviews.csv', convert=True, chunk_size = 1_000_000)

In [61]:
test1['timestamp_created']=test1['timestamp_created'].values.astype('datetime64[D]')

AttributeError: 'pyarrow.lib.StringArray' object has no attribute 'astype'

In [ ]:
dates_input = df["month_15"].values.astype('datetime64[D]')
testf(df['month_15'].astype('datetime64[D]').values)

In [13]:
vaex_df.info()

column,type,unit,description,expression
Unnamed: 0,int64,,,
app_id,int64,,,
app_name,str,,,
review_id,int64,,,
language,str,,,
review,str,,,
timestamp_created,int64,,,
timestamp_updated,int64,,,
recommended,bool,,,
votes_helpful,int64,,,


In [78]:
# Export the dataset and store it as an hdf5 file

#vaex_df.export('vaex_file_test.hdf5')

In [8]:
# Open the hdf5 file
vaex_df = vaex.open('steam_reviews.csv.hdf5')

In [81]:
vaex_df['timestamp_created']=datetime.utcfromtimestamp(vaex_df.timestamp_created).strftime('%Y-%m-%d %H:%M:%S')

TypeError: an integer is required (got type Expression)

In [14]:
def column_to_datetime(datetime_str):
    #return np.datetime64(datetime.strptime(datetime_str, date_format))
    return datetime.utcfromtimestamp(datetime_str).strftime('%Y-%m-%d %H:%M:%S')

vaex_df['timestamp_created'] = vaex_df['timestamp_created'].apply(column_to_datetime)

In [15]:
vaex_df.info()

column,type,unit,description,expression
Unnamed: 0,int64,,,
app_id,int64,,,
app_name,str,,,
review_id,int64,,,
language,str,,,
review,str,,,
timestamp_created,virtual column,,,lambda_function(__timestamp_created)
timestamp_updated,int64,,,
recommended,bool,,,
votes_helpful,int64,,,


In [ ]:
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

In [17]:
# Confirm we get 315 unique values from the vaex dataframe
vaex_df.app_id.nunique()

315

In [48]:
test1.info()

column,type,unit,description,expression
Unnamed: 0,int64,,,
app_id,int64,,,
app_name,str,,,
review_id,int64,,,
language,str,,,
review,str,,,
timestamp_created,str,,,
timestamp_updated,str,,,
recommended,bool,,,
votes_helpful,int64,,,


In [49]:
# -- Way 1
#test1['timestamp_created']=test1.timestamp_created.values.astype('datetime64[ns]')
#AttributeError: 'pyarrow.lib.StringArray' object has no attribute 'astype'

# -- Way 2
#from datetime import datetime
#date_format = "%m/%d/%Y"

#def column_to_datetime(datetime_str):
    #return np.datetime64(datetime.strptime(datetime_str, date_format))
    #return datetime.utcfromtimestamp(datetime_str).strftime('%Y-%m-%d %H:%M:%S')

#test1['timestamp_created'] = test1['timestamp_created'].apply(column_to_datetime)
#Error is the timestamp does not math the date format that we're using


# -- Way 3
# Timestamps originally read as int64. This works on int
vaex_df['timestamp_created']=datetime.utcfromtimestamp(vaex_df.timestamp_created).strftime('%Y-%m-%d %H:%M:%S')
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))



vaex_df['pDate']=vaex_df.Date.values.astype('datetime64[ns]')

AttributeError: 'pyarrow.lib.StringArray' object has no attribute 'astype'

In [82]:
prueba1 = 1611381629
type(prueba1)

int

In [83]:
print(datetime.utcfromtimestamp(prueba1).strftime('%Y-%m-%d %H:%M:%S'))

2021-01-23 06:00:29


In [86]:
vaex_df['timestamp_created'].min()

array(1290197836)

In [71]:
test1.info()

ERROR:MainThread:vaex:error evaluating: timestamp_created at rows 0-5
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/brauliovillalobos/opt/anaconda3/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/brauliovillalobos/opt/anaconda3/lib/python3.8/site-packages/vaex/expression.py", line 1614, in _apply
    scalar_result = self.f(*[fix_type(k[i]) for k in args], **{key: value[i] for key, value in kwargs.items()})
  File "<ipython-input-70-09bc19e09c32>", line 6, in column_to_datetime
TypeError: an integer is required (got type str)
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/brauliovillalobos/opt/anaconda3/lib/python3.8/site-packages/vaex/dataframe.py", line 3865, in table_part
    values[name] = df.evaluate(name)
  File "/Users/brauliovillalobos/opt/anaconda3/lib/python3.8/site-packages/vaex/datafra

column,type,unit,description,expression
Unnamed: 0,int64,,,
app_id,int64,,,
app_name,str,,,
review_id,int64,,,
language,str,,,
review,str,,,
timestamp_created,virtual column,,,lambda_function(__timestamp_created)
timestamp_updated,str,,,
recommended,bool,,,
votes_helpful,int64,,,


# RQ1: Exploratory Data Analysis (EDA)

Our dataset to analyze is composed of 23 columns or features and 21.747.371 observations. 

In [19]:
vaex_df.shape

(21747371, 23)

With the info command we identify the types of variables. 


Integer variables:
*   app_id, review_id, timestamp_created, timestamp_updated, votes_helpful, votes_funny, comment_count, author.steamid, author.num_gaes_owned, author.num_reviews

String variables:
*   app_name, language, review

Float variables:
*   weighted_vote_score, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played

Boolean variables:

*   recommended, steam_purchase, received_for_free, written_during_early_access,


In [37]:
vaex_df.info()

column,type,unit,description,expression
Unnamed: 0,int64,,,
app_id,int64,,,
app_name,str,,,
review_id,int64,,,
language,str,,,
review,str,,,
timestamp_created,int64,,,
timestamp_updated,int64,,,
recommended,bool,,,
votes_helpful,int64,,,


Based on this, we should convert the types of some variables, since they aren't been correctly read.

* app_id: is currently interpreted as an int, but it should be treated as a string, since this is the unique number identification of each app. 
* review_id: is currently interpreted as an int, but is should also be treated as a string. 
* timestamp_created & timestamp_updated: currently interpreted as int, but they should be treated as datetime objects. 

In [42]:
vaex_df['app_id']= vaex_df.app_id.astype('str')
vaex_df['review_id']= vaex_df.review_id.astype('str')
vaex_df['timestamp_created']=datetime.utcfromtimestamp(vaex_df.inttimestamp_created).strftime('%Y-%m-%d %H:%M:%S')
vaex_df['timestamp_updated']=vaex_df.timestamp_updated.astype('datetime64[ns]')

TypeError: an integer is required (got type Expression)

In [43]:
vaex_df.timestamp_created= vaex_df.timestamp_created.apply(lambda d: datetime.datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))

#datetime.utcfromtimestamp(vaex_df.timestamp_created).strftime('%Y-%m-%d %H:%M:%S')

#df.date = df.date.apply(lambda d: datetime.datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))

In [45]:
vaex_df.timestamp_created = vaex_df.timestamp_created.apply(lambda d: datetime.strftime(d,'%Y-%m-%d'))

In [ ]:
df.date.apply(lambda d: datetime.strptime(d, "%Y-%m-%d"))

In [ ]:
df.date = df.date.apply(lambda d: datetime.datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))

In [46]:
vaex_df.info()

column,type,unit,description,expression
Unnamed: 0,int64,,,
app_id,virtual column,,,"astype(__app_id_1, 'str')"
app_name,str,,,
review_id,virtual column,,,"astype(__review_id_1, 'str')"
language,str,,,
review,str,,,
timestamp_created,int64,,,
timestamp_updated,int64,,,
recommended,bool,,,
votes_helpful,int64,,,


Description of the numerical values: 

In [39]:
vaex_df[:,1:].describe()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
data_type,int64,string,int64,string,large_string,int64,int64,bool,int64,int64,...,bool,bool,bool,int64,int64,int64,float64,float64,float64,float64
count,21747371,21747371,21747371,21747371,21713629,21747371,21747371,21747371,21747371,21747371,...,21747371,21747371,21747371,21747371,21747371,21747371,21747369,21747369,21721689,21747369
NA,0,0,0,0,33742,0,0,0,0,0,...,0,0,0,0,0,0,2,2,25682,2
mean,392818.0819021297,--,51874995.36142856,--,--,1544432473.6402824,1547555664.105695,0.8747084877523816,404468.8552777713,126791.72557597881,...,0.7744583011896012,0.03163881280178648,0.09271856354499125,76494792465.02202,1011300.0967205645,404477.4800190791,16091.04968789558,155.5421350968938,8807.421419761604,1580505062.0580525
std,248097.734731,--,20842672.266399,--,--,58121936.913771,56898990.858622,0.331049,1333740730.362596,23335525.369373,...,0.417939,0.175037,0.290038,88399671708.809021,2108829112.611605,1333740730.360435,37430.566898,730.048788,23885.529993,46761774.154387
min,70,--,43,--,--,1290197836,1290197836,False,0,0,...,False,False,False,76561197960265728,0,1,0.0,0.0,1.0,0.0
max,1291340,--,85218673,--,--,1611426287,2283382797,True,4398046511106,4294967295,...,True,True,True,76561199133265376,4398046511619,4398046511108,3744943.0,27039.0,3228103.0,1611434142.0


In [ ]:
!ls -a